In [142]:
import psycopg2
import sys
import os
import pandas as pd

In [329]:
HOST="htm-redhorse.cpgyjyre5uq9.us-west-2.rds.amazonaws.com"
DATABASE="htm_redhorse_postgres"
USER="westinn"
PASSWORD="rootroot"


In [388]:
#Define our connection string
conn_string = "host='"+ HOST + "' dbname='" + DATABASE +"' user='" + USER + "' password='" + PASSWORD +"'"
 
# print the connection string we will use to connect
print("Connecting to database\n	->%s" % (conn_string))
 
# get a connection, if a connect cannot be made an exception will be raised here
conn = psycopg2.connect(conn_string)
 
# conn.cursor will return a cursor object, you can use this cursor to perform queries
cursor = conn.cursor()
print("Connected!\n")


def execute(query, cursor=cursor):
    try:
        print("Query:")
        print(query)
        cursor.execute(query)
        rows = cursor.fetchall()
        return rows
    except Exception as err:
        print("Error:")
        print(err)
        cursor.execute("rollback")
        

Connecting to database
	->host='htm-redhorse.cpgyjyre5uq9.us-west-2.rds.amazonaws.com' dbname='htm_redhorse_postgres' user='westinn' password='rootroot'
Connected!



In [327]:
# helper functions
def get_csvs(directory):
    csv_files = []
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                csv = os.path.join(subdir,file)
                csv_files.append(csv)
                print(file)
    return csv_files

def create_polygon(min_LAT, max_LAT, min_LON, max_LON):
    polygon_string = "LINESTRING({})"
    polygon_points = "{min_LON} {min_LAT}, {min_LON} {max_LAT}, {max_LON} {max_LAT}, {max_LON} {min_LAT}, {min_LON} {min_LAT}"
    polygon_points = polygon_points.format(min_LAT=min_LAT, max_LAT=max_LAT, min_LON=min_LON, max_LON=max_LON)
    polygon_string = polygon_string.format(polygon_points)
    return polygon_string

def create_linestring(points_arry): # [[lon, lat], [lon, lat]]
    line_string = "LINESTRING({})"
    line_points = ""
    for idx in range(0,len(points_arry)):
        line_points += "{} {}, ".format(points_arry[idx][0], points_arry[idx][1])
    line_points = line_points[:-2]
    line_string = line_string.format(line_points)
    return line_string

In [299]:
query = """
SELECT * FROM ais LIMIT 50
"""

rows = execute(query)
print(rows)

Query:

SELECT * FROM ais LIMIT 50

[(Decimal('367455580'), datetime.datetime(2017, 1, 15, 15, 17, 21), 27.9105, -82.44117, Decimal('6.6'), Decimal('153.0'), Decimal('153.0'), None, None, None, None, 'moored', None, None, Decimal('12.2'), None), (Decimal('367427760'), datetime.datetime(2017, 1, 15, 15, 17, 25), 32.03464, -80.90286, Decimal('0.0'), Decimal('0.0'), Decimal('511.0'), None, None, None, None, None, None, None, None, None), (Decimal('338131948'), datetime.datetime(2017, 1, 15, 15, 17, 43), 30.33339, -81.61214, Decimal('0.0'), Decimal('92.1'), Decimal('511.0'), None, None, None, None, None, None, None, None, None), (Decimal('367703150'), datetime.datetime(2017, 1, 15, 15, 17, 47), 28.23862, -82.74965, Decimal('0.0'), Decimal('186.0'), Decimal('511.0'), None, None, None, None, None, None, None, None, None), (Decimal('339300320'), datetime.datetime(2017, 1, 15, 15, 17, 49), 24.8894, -79.67337, Decimal('16.8'), Decimal('-64.7'), Decimal('348.0'), None, None, None, None, 'under w

In [116]:
#directory = r'C:\Users\DennisSilva\Documents\HTM\data\upload'
directory = r'C:\Users\DennisSilva\Documents\HTM\data\AIS_2017_01_Zone17\AIS_ASCII_by_UTM_Month\2017'
csv_files = get_csvs(directory)

copy_sql = """
           COPY ais FROM stdin WITH CSV HEADER
           DELIMITER as ','
           """
for csv in csv_files:
    with open(csv, 'r') as f:
        cursor.copy_expert(sql=copy_sql, file=f)
        conn.commit()

AIS_2017_01_Zone17.csv


In [370]:
# get distinct MMSI
query = """
SELECT DISTINCT "MMSI" FROM ais
"""
rows = execute(query)
print(rows)

Query:

SELECT DISTINCT "MMSI" FROM ais

[(378111841,), (367602140,), (319060200,), (316029994,), (367610560,), (366766840,), (338235061,), (367306040,), (367733050,), (367667720,), (367742880,), (338923000,), (257366000,), (367412560,), (538005776,), (367343230,), (636015412,), (367135170,), (367115360,), (355833000,), (319999000,), (366960160,), (367115580,), (538003095,), (319887000,), (338131333,), (316023269,), (339303000,), (338133032,), (367761890,), (477848500,), (565174000,), (367719010,), (369970042,), (319420000,), (319370000,), (367501020,), (367696730,), (367014880,), (366919110,), (258251000,), (378112112,), (246456000,), (338017000,), (367658180,), (338205826,), (368922000,), (636015295,), (338209433,), (371583000,), (416481000,), (224517240,), (367688630,), (338192290,), (538070899,), (338004587,), (338183198,), (338178129,), (366836950,), (367541650,), (257325000,), (367701880,), (369970517,), (367595560,), (636017280,), (636016606,), (316023808,), (319767000,), (36755

In [389]:
import pandas as pd
failed_mmsi = []
for idx,row in enumerate(rows): # for each mmsi
    #try:
        if idx > 0:
            break
        tup = []
        
        mmsi = int(row[0])

        query = """
        SELECT * from ais
        WHERE "MMSI"={}
        """.format(mmsi)

        df = pd.io.sql.read_sql_query(query, conn) # get all rows with mmsi

        grouped = df.groupby(pd.Grouper(key='BaseDate', freq='1H')) # group by hour

        for key, df_group in grouped: # for each group, make one row for analysis table
            if not df_group.empty and df_group.shape[0] > 1:
                MMSI = int(df_group['MMSI'].min())
                min_LAT = df_group['LAT'].min()
                max_LAT = df_group['LAT'].max()
                min_LON = df_group['LON'].min()
                max_LON = df_group['LON'].max()
                min_BaseDate = df_group['BaseDate'].min()
                max_BaseDate = df_group['BaseDate'].max()
                min_hour_BaseDate = df_group['BaseDate'].min().replace(microsecond=0,second=0,minute=0)
                min_SOG = df_group['SOG'].min()
                max_SOG = df_group['SOG'].max()
                
                df_group = df_group.sort_values(by="BaseDate")
                points_arry = [list(t) for t in zip(df_group['LON'], df_group['LAT'])]
                geometry = create_linestring(points_arry)
                
                print(geometry)
                print()

                new_row = (MMSI, 
                           min_LAT, max_LAT, 
                           min_LON, max_LON, 
                           min_BaseDate, max_BaseDate, 
                           min_hour_BaseDate,
                           min_SOG, max_SOG, 
                           geometry,
                           points_arry)

                tup.append(new_row)

        # 4 nautical miles is 7408 meters
        cursor.executemany("INSERT INTO analysis VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, ST_Buffer(ST_GeomFromText(%s,4326), 7408), %s)", tup)
         #cursor.executemany("INSERT INTO analysis VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, ST_Polygon(ST_GeomFromText(%s),4326))", tup)
       
        
        print(mmsi)
        print(round(idx/len(rows),2))
        print()
        
#     except Exception as err:
#         failed_mmsi.append(mmsi)
#         print(mmsi)
#         print("Error: {}".format(err))
#         print()
        


LINESTRING(-80.1224 26.11089, -80.12238 26.1109)

LINESTRING(-80.12238 26.11091, -80.12239 26.11091, -80.1224 26.11087)

LINESTRING(-80.12239 26.11087, -80.12238 26.11088)

LINESTRING(-80.1224 26.11087, -80.1224 26.1109)

LINESTRING(-80.1224 26.1109, -80.12239 26.11089, -80.12239 26.11089, -80.12237 26.11091, -80.12237 26.11091, -80.12243 26.11095, -80.12246 26.11097, -80.12247 26.11096)

LINESTRING(-80.12239 26.11088, -80.12016 26.11027, -80.12044 26.10874, -80.12047 26.10764, -80.12026 26.1061, -80.11956 26.10345)

LINESTRING(-80.12171 26.10275, -80.12248 26.10271, -80.12393 26.10275, -80.12505 26.10255, -80.12508 26.10273, -80.12421 26.10281, -80.12251 26.1028, -80.12252 26.1029, -80.12253 26.10317, -80.12253 26.10316)

378111841
0.0



In [387]:
tup

[(378111841,
  26.11089,
  26.1109,
  -80.1224,
  -80.12238,
  Timestamp('2017-01-24 14:41:31'),
  Timestamp('2017-01-24 14:54:32'),
  Timestamp('2017-01-24 14:00:00'),
  0.0,
  0.0,
  'LINESTRING(-80.1224 26.11089, -80.12238 26.1109)',
  [[-80.1224, 26.11089], [-80.12238, 26.1109]]),
 (378111841,
  26.11087,
  26.11091,
  -80.1224,
  -80.12238,
  Timestamp('2017-01-24 15:15:36'),
  Timestamp('2017-01-24 15:45:32'),
  Timestamp('2017-01-24 15:00:00'),
  0.0,
  0.1,
  'LINESTRING(-80.12238 26.11091, -80.12239 26.11091, -80.1224 26.11087)',
  [[-80.12238, 26.11091], [-80.12239, 26.11091], [-80.1224, 26.11087]]),
 (378111841,
  26.11087,
  26.11088,
  -80.12239,
  -80.12238,
  Timestamp('2017-01-24 16:00:30'),
  Timestamp('2017-01-24 16:21:33'),
  Timestamp('2017-01-24 16:00:00'),
  0.1,
  0.1,
  'LINESTRING(-80.12239 26.11087, -80.12238 26.11088)',
  [[-80.12239, 26.11087], [-80.12238, 26.11088]]),
 (378111841,
  26.11087,
  26.1109,
  -80.1224,
  -80.1224,
  Timestamp('2017-01-26 15:24:

In [374]:
df_group

,MMSI,BaseDate,LAT,LON,SOG,COG,HEADING,VesselName,IMO,Callsign,VesselType,Status,Length,Width,Draft,Cargo
22,378111841,2017-01-30 15:01:12,26.10275,-80.12171,2.3,-138.1,511.0,None,None,None,None,None,None,None,None,None
23,378111841,2017-01-30 15:02:15,26.10271,-80.12248,2.3,-145.0,511.0,None,None,None,None,None,None,None,None,None
24,378111841,2017-01-30 15:04:13,26.10275,-80.12393,2.4,-152.3,511.0,None,None,None,None,None,None,None,None,None
25,378111841,2017-01-30 15:07:13,26.10255,-80.12505,0.1,156.5,511.0,None,None,None,None,None,None,None,None,None
33,378111841,2017-01-30 15:08:44,26.10273,-80.12508,0.1,151.3,511.0,None,None,None,None,None,None,None,None,None
26,378111841,2017-01-30 15:10:13,26.10281,-80.12421,3.0,85.4,511.0,None,None,None,None,None,None,None,None,None
27,378111841,2017-01-30 15:12:42,26.10280,-80.12251,1.8,-67.2,511.0,None,None,None,None,None,None,None,None,None
28,378111841,2017-01-30 15:14:42,26.10290,-80.12252,0.1,154.5,511.0,None,None,None,None,None,None,None,None,None
29,378111841,2017-01-30 15:16:46,26.10317,-80.12253,0.1,141.0,511.0,None,None,None,None,None,None,None,None,None
30,378111841,2017-01-30 15:18:14,26.10316,-80.12253,0.1,145.2,511.0,None,None,None,None,None,None,None,None,None


In [335]:
import pandas as pd
failed_mmsi = []
for idx,row in enumerate(rows): # for each mmsi
    try:
        tup = []
        
        mmsi = int(row[0])
        
        if idx > 2:
            break

        query = """
        SELECT * from ais
        WHERE "MMSI"={}
        """.format(mmsi)

        df = pd.io.sql.read_sql_query(query, conn) # get all rows with mmsi

        grouped = df.groupby(pd.Grouper(key='BaseDate', freq='1H')) # group by hour

        for key, df_group in grouped: # for each group, make one row for analysis table
            if not df_group.empty:
                MMSI = df_group['MMSI'].min()
                min_LAT = df_group['LAT'].min()
                max_LAT = df_group['LAT'].max()
                min_LON = df_group['LON'].min()
                max_LON = df_group['LON'].max()
                min_BaseDate = df_group['BaseDate'].min()
                max_BaseDate = df_group['BaseDate'].max()
                min_hour_BaseDate = df_group['BaseDate'].min().replace(microsecond=0,second=0,minute=0)
                min_SOG = df_group['SOG'].min()
                max_SOG = df_group['SOG'].max()
                
                df_group = df_group.sort_values(by="BaseDate")
                points_arry = [list(t) for t in zip(df_group['LON'], df_group['LAT'])]
                geometry = create_linestring(points_arry)

                new_row = (MMSI, 
                           min_LAT, max_LAT, 
                           min_LON, max_LON, 
                           min_BaseDate, max_BaseDate, 
                           min_hour_BaseDate,
                           min_SOG, max_SOG, 
                           geometry,
                           points_arry)

                tup.append(new_row)


        #cursor.executemany("INSERT INTO analysis VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, ST_Buffer(ST_GeomFromText(%s),4326)), %s", tup)
        
        print(mmsi)
        print(round(idx/len(rows),2))
        print()
        
    except Exception as err:
        failed_mmsi.append(mmsi)
        print(mmsi)
        print("Error: {}".format(err))
        print()
        


(338131948,
 30.33336,
 30.33337,
 -81.61214,
 -81.61214,
 Timestamp('2017-01-16 17:02:44'),
 Timestamp('2017-01-16 17:08:44'),
 Timestamp('2017-01-16 17:00:00'),
 0.0,
 0.0,
 'LINESTRING(-81.61214 30.33337, -81.61214 30.33336, -81.61214 30.33336)')

In [390]:
query = """
SELECT * FROM analysis LIMIT 50
"""

rows = execute(query)
print(rows)

Query:

SELECT * FROM analysis LIMIT 50

[(378111841, Decimal('26.11089'), Decimal('26.1109'), Decimal('-80.1224'), Decimal('-80.12238'), datetime.datetime(2017, 1, 24, 14, 41, 31), datetime.datetime(2017, 1, 24, 14, 54, 32), datetime.datetime(2017, 1, 24, 14, 0), Decimal('0.0'), Decimal('0.0'), '0103000020E610000001000000220000003B0B0A512982AAC0A46A3C0C07FCB9404938A36C15D39FC0B57378DB0903BC4028EFFF5B25EA82C0743BBE5279F7BC40832E5EF152738A408C0B99B4F0CFBC40D777A75EFCACA140086407EFF48DBB40AC09CF001709AC407D0F37A9E53DB940940BD8B39CA5B24064CD598883F6B5403C0E072C278CB64002DA275910D8B140D577FF53CB91B9401E2A55702316AA40E2803B23CE98BB40477639AB09FB9E40A448819A3D8DBC40286B2CD90D3A8140BF185CFCB465BC4083B231746A238CC03F71CA36B923BB40D7585C3F0219A2C0B51CFAF0A9D3B840AFEA83E11C75ACC09FDA1CD0478CB540187C32A49FDBB2C03EE7EAA0D46DB140C27E611C2AC2B6C07F44DBFFAB41A94064E85944CEC7B9C0787D07C51A529D4020B73D14D1CEBBC00CF3901960D07B40E07E838B40C3BCC023A4954048758FC0F84E5EEDB79BBCC0405575B279EDA2C075A7CC27BC59

In [ ]:
query = """
INSERT INTO test VALUES (42, 'tom brady')
"""
print(query)

rows = execute(query)
print(rows)